# QA with reference

We believe that hallucinations pose a major problem in the adoption of LLMs (Language Model Models). It is imperative to provide a simple and quick solution that allows the user to verify the coherence of the answers to the questions they are asked.

The conventional approach is to provide a list of URLs of the documents that helped in answering (see qa_with_source). However, this approach is unsatisfactory in several scenarios:

1. The question is asked about a PDF of over 100 pages. Each fragment comes from the same document, but from where?
2. Some documents do not have URLs (data retrieved from a database or other loaders).

It appears essential to have a means of retrieving all references to the actual data sources used by the model to answer the question. 

This includes:
- The precise list of documents used for the answer (the `Documents`, along with their metadata that may contain page numbers, slide numbers, or any other information allowing the retrieval of the fragment in the original document).
- The excerpts of text used for the answer in each fragment. Even if a fragment is used, the LLM only utilizes a small portion to generate the answer. Access to these verbatim excerpts helps to quickly ascertain the validity of the answer.

We propose a new pipeline: `qa_with_reference` for this purpose. It is a Question/Answer type pipeline that returns the list of documents used, and in the metadata, the list of verbatim excerpts exploited to produce the answer.

*At this time, only the `map_reduce` chain type car extract the verbatim excerpts.*


In [ ]:
#!pip install 'langchain-qa_with_references[openai]'

In [7]:
from langchain import OpenAI
from langchain.schema import Document
llm = OpenAI(
            temperature=0,
            max_tokens=1000,
        )

In [3]:

from langchain_qa_with_references.chains import QAWithReferencesAndVerbatimsChain
chain_type="map_reduce"  # Only map_reduce can extract the verbatim.
qa_chain = QAWithReferencesAndVerbatimsChain.from_chain_type(
        llm=llm,
        chain_type=chain_type,
    )

question = "what does it eat?"
bodies = ["he eats apples and plays football."
          "My name is Philippe."
          
          "he eats pears.",
          
          "he eats carrots. I like football.",
          "The Earth is round."
]
docs=[Document(page_content=body,metadata={"id":i}) for i,body in enumerate(bodies)]

answer = qa_chain(
        inputs={
            "docs": docs,
            "question": question,
        },
    )


print(f'To answer "{answer["answer"]}", the LLM use:')
for doc in answer["source_documents"]:
    print(f"Document {doc.metadata['id']}")
    for verbatim in doc.metadata.get("verbatims",[]):
        print(f'- "{verbatim}"')


To answer "He eats apples, pears and carrots.", the LLM use:
Document 0
- "he eats apples"
- "he eats pears."
Document 1
- "he eats carrots."


In [ ]:
#!pip install chroma wikipedia

In [9]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import WikipediaRetriever
from langchain.vectorstores import Chroma

wikipedia_retriever = WikipediaRetriever()
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(),
    persist_directory="/tmp/chroma_db_oai",
)
docs = wikipedia_retriever.get_relevant_documents(question)
from langchain.text_splitter import RecursiveCharacterTextSplitter

split_docs = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=10
).split_documents(docs)

vectorstore.add_documents(split_docs)
retriever = vectorstore.as_retriever()

question = "what can you say about ukraine?"

In [10]:
from langchain_qa_with_references.chains import RetrievalQAWithReferencesAndVerbatimsChain
from typing import Literal
chain_type:Literal["stuff","map_reduce","map_rerank","refine"]="map_reduce"

qa_chain = RetrievalQAWithReferencesAndVerbatimsChain.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=retriever,
    reduce_k_below_max_tokens=True,
)
answer = qa_chain(
    inputs={
        "question": question,
    }
)
print(f'For the question "{question}", to answer "{answer["answer"]}", the LLM use:')
for doc in answer["source_documents"]:
    print(f"Source {doc.metadata['source']}")
    for verbatim in doc.metadata.get("verbatims", []):
        print(f'-  "{verbatim}"')

For the question "what can you say about ukraine?", to answer "Ukraine's ethnocentrism, Ukraine's existence, Ukraine should be dismantled, Ukraine cannot be allowed to exist, the brutal censorship of the Ukrainian culture, large-scale de-ukrainization of Ukrainians, Ukraine's ethnocentrism is an artificial perversion, Ukraine's existence is impossible as a nation-state, the word 'Ukraine' itself cannot be allowed to exist, Ukraine should be dismantled and replaced with several states, the Ukrainian people's right to self-determination would not be infringed upon", the LLM use:
Source https://en.wikipedia.org/wiki/Russo-Ukrainian_War
-  "Ukraine."
Source https://en.wikipedia.org/wiki/What_Russia_Should_Do_with_Ukraine
-  "Ukraine's ethnocentrism"
-  "Ukraine's existence"
-  "Ukraine should be dismantled"
Source https://en.wikipedia.org/wiki/What_Russia_Should_Do_with_Ukraine
-  "brutal censorship" of the Ukrainian culture"
-  "large-scale "de-ukrainization" of Ukrainians"
-  "Ukraine's 